In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from keras.models import load_model
model = load_model('drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small_2.h5')
model.summary()

img_path = 'drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small/test/cats/cat.1700.jpg'
#from keras.preprocessing import image
import keras.utils as image
import numpy as np
img = image.load_img(img_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.
print(img_tensor.shape)

import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()

## Dog and Cat
# https://www.kaggle.com/c/dogs-vs-cats/data

# 디렉토리 구조
# trian - cats / dogs
# validation - cats / dogs
# test - cats / dogs

In [ ]:
# prepare dataset
import os, shutil

base_dir = './drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small'
train_dir = './drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small/train'
validation_dir = './drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small/validation'
test_dir = './drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small/test'

# create neural network
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import models
model = models.Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3))) # 필터 32 (3,3) / 입력 150 x 150 RGB 이미지
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 개와 고양이 구분
model.summary()

In [ ]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

# Data Preprocessing
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
 train_dir,
 target_size=(150,150),
 batch_size=20,
 class_mode='binary'
)
validation_generator = test_datagen.flow_from_directory(
 validation_dir,
 target_size=(150,150),
 batch_size=20,
 class_mode='binary'
)

for data_batch, labels_batch in train_generator:
  print ('배치 데이터 크기:', data_batch.shape)
  print ('배치 레이블 크기:', labels_batch.shape)
  break

history = model.fit_generator( # 배치 제너레이터를 사용하여 모델 훈련
 train_generator,
 steps_per_epoch=100,
 epochs=30,
 validation_data=validation_generator,
 validation_steps=50
)
model.save('drive/MyDrive/cats_and_dogs_small/cats_and_dogs_small_1.h5') # 모델 저장

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc)+1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
from keras import models
layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

activations = activation_model.predict(img_tensor)
first_layer_activation = activations[0]
print (first_layer_activation.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.matshow(first_layer_activation[0, :, :, 19], cmap='viridis')

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 15], cmap='viridis')

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 0], cmap='viridis')